In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df = pd.read_csv('company_data.csv')
df


,TICKER,COMPANY_NAME,TOTAL_SHARES,SIR,RSI,BF,ADV,PUBLIC
0,AAPL,Apple Inc.,7135253148,0.29,55.5,0.016,59608521,0.59
1,MSFT,Microsoft Corporation,347596351,0.16,65.2,0.006,14908738,0.64
2,GOOG,Alphabet Inc. (Google),284799140,0.14,59.6,0.010,37050339,0.27
3,AMZN,Amazon.com Inc.,1356654201,0.24,54.5,0.012,41844587,0.58
4,NVDA,NVIDIA Corporation,3376062168,0.29,59.5,0.020,24500947,0.56
5,META,Meta Platforms Inc. (Facebook),522054010,0.16,44.0,0.007,19382047,0.54
6,TSLA,Tesla Inc.,1324570934,0.25,38.0,0.015,36293482,0.53
7,BRK.B,Berkshire Hathaway Inc.,2012345678,0.12,48.0,0.008,9000012,0.38
8,JPM,JPMorgan Chase & Co.,703473827,0.19,51.2,0.009,12004980,0.49
9,V,Visa Inc.,989234573,0.17,46.7,0.010,30451238,0.42


In [ ]:
class Person:
    def __init__(self, id, group):
        self.id = id
        self.group = group

    # def decide(self, company_state):
    #     if self.group == "risky":
    #         if company_state['SIR'] > 0.3 and company_state['RSI'] < 40:
    #             return 1e  # Buy
    #         else:
    #             return 0  # Hold
    #     elif self.group == "scared":
    #         if company_state['PUBLIC'] > 0.5 or company_state['RSI'] > 40:
    #             return -1  # Sell
    #         else:
    #             return 0  # Hold
    #     else:  # neutral
    #         return 0

    def decide(self, company_state):
        if self.group == "risky":
            w = {'SIR': 0.3, 'RSI': 0.25, 'BF': 0.05, 'ADV': 0.1, 'PUBLIC': 0.3 }
            T1, T2 = -0.25, 0.1

        elif self.group == "neutral":
            w = {'SIR': 0.25, 'RSI': 0.2, 'BF': 0.15, 'ADV': 0.1, 'PUBLIC': 0.3 }
            T1, T2 = -0.1, 0.25

        else:
            w = {'SIR': 0.1, 'RSI': 0.05, 'BF': 0.3, 'ADV': 0.2, 'PUBLIC': 0.35 }
            T1, T2 = 0.05, 0.45

        # some vars need to be scaled for their effect on the D to be similar and for the weights to make sense
        RSI = company_state['RSI']/100
        BF = company_state['BF']/0.05 # assuming that the max fee is 5%
        ADV = company_state['ADV'] / company_state['TOTAL_SHARES']
        D = (w['SIR'] * company_state['SIR'] + w['RSI'] * RSI + w['BF']* BF + w['ADV']* ADV + w['PUBLIC']* company_state['PUBLIC'])
        #print(f"D = ",D)
        if D < T1:
            return -1   # Sell
        elif D > T2:
            return 1    # Buy
        else:
            return 0    # Hold      

def assign_shares_exact(total_shares, N):
    weights = np.random.random(N)
    weights /= weights.sum()
    shares = np.floor(weights * total_shares).astype(int)
    leftover = total_shares - shares.sum()
    if leftover > 0:
        indices = np.random.choice(N, leftover, replace=False)
        shares[indices] += 1
    assert shares.sum() == total_shares
    return shares

N = 100  # Persons total

company_data = pd.read_csv('company_data_revised.csv')
for idx, company_state in company_data.iterrows():
    company_name = company_state['COMPANY_NAME']
    buys = 0
    sells = 0

    persons = [Person(i, np.random.choice(['risky', 'scared', 'neutral'])) for i in range(N)]

    total_shares = company_state['TOTAL_SHARES']
    shares_per_person = assign_shares_exact(total_shares, N)

    decisions = []

    for i, person in enumerate(persons):
        decision = person.decide(company_state)
        decisions.append([person.id, person.group, decision, shares_per_person[i]])
        buys += 1 if decision == 1 else 0
        sells += 1 if decision == -1 else 0


    # df_decisions = pd.DataFrame(decisions, columns=['ID', 'GROUP', 'DECISION', 'SHARES'])
    # filename = f"{company_name}.csv"
    # df_decisions.to_csv(filename, index=False)
    
    ## CHECKLING IF SUM OF SHARE == TOTAL SHARES 
    print(f"The sum of all share are : ",sum(shares_per_person))
    print(f"the number of shares that should match the total_shares are :",total_shares)
    print("YAY") if sum(shares_per_person) == total_shares else print("DAMN")
    # info 
    print(f"Company: {company_name}")
    print(f"The number of buys are ", buys)
    print(f"The number of sells are ", sells)
    holds = N - buys - sells
    print(f"The number of holds are ", holds)


The sum of all share are :  7135253148
the number of shares that should match the total_shares are : 7135253148
YAY
Company: Apple Inc.
The number of buys are  100
The number of sells are  0
The number of holds are  0
The sum of all share are :  347596351
the number of shares that should match the total_shares are : 347596351
YAY
Company: Microsoft Corporation
The number of buys are  74
The number of sells are  0
The number of holds are  26
The sum of all share are :  284799140
the number of shares that should match the total_shares are : 284799140
YAY
Company: Alphabet Inc. (Google)
The number of buys are  67
The number of sells are  0
The number of holds are  33
The sum of all share are :  1356654201
the number of shares that should match the total_shares are : 1356654201
YAY
Company: Amazon.com Inc.
The number of buys are  100
The number of sells are  0
The number of holds are  0
The sum of all share are :  3376062168
the number of shares that should match the total_shares are : 337